## Manage Database Credentials using GCP Secret Manager

Here is the process involved to get secret details as part of the applications.
* Create Secret Manager Client Object
* Get Secret Details
* Use Secret Details (to connect to Databases)

In [ ]:
from google.cloud import secretmanager

In [ ]:
client = secretmanager.SecretManagerServiceClient()

In [ ]:
client.access_secret_version?

In [ ]:
project_id = 'tidy-fort-361710'
secret_id = 'retailsecret'
version_id = 1

In [ ]:
secret_name = f'projects/{project_id}/secrets/{secret_id}/versions/{version_id}'

In [ ]:
secret_name

In [ ]:
response = client.access_secret_version(name=secret_name)

In [ ]:
type(response)

In [ ]:
response.payload

In [ ]:
response.payload.data

In [ ]:
response.payload.data.decode('utf-8')

In [ ]:
import json

In [ ]:
secret_details = json.loads(response.payload.data.decode('utf-8'))

In [ ]:
secret_details.keys()

In [ ]:
secret_details['user']

In [ ]:
secret_details['password']

In [ ]:
import json
from google.cloud import secretmanager

def get_secret_details(secret_name):
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(name=secret_name)
    return json.loads(response.payload.data.decode('utf-8'))

In [ ]:
project_id = 'tidy-fort-361710'
secret_id = 'retailsecret'
version_id = 1

secret_name = f'projects/{project_id}/secrets/{secret_id}/versions/{version_id}'
secret_details = get_secret_details(secret_name)

In [ ]:
secret_details

In [ ]:
conn_uri = 'postgresql://{user}:{password}@{host}:{port}/{database}'

In [ ]:
conn_uri.format(port=5432, **secret_details)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_sql(
    '''
        SELECT order_status, sum(order_count) AS order_count FROM daily_status_count
        GROUP BY 1
        ORDER BY 2 DESC
    ''',
    conn_uri.format(port=5432, **secret_details)
)

In [ ]:
df